In [ ]:
import time
import os
import numpy as np
from acpreprocessing.utils.metadata import parse_metadata
from acpreprocessing.stitching_modules.convert_to_n5 import acquisition_dir_to_n5_dir
from acpreprocessing.utils.nglink import update_state, create_overview
from acpreprocessing.stitching_modules.stitch import create_json, stitch
import argschema
from argschema.fields import Boolean, Int, Str
import posixpath

: 

In [ ]:
#acq_id = 'MN16_RH_S26a_220908_overview'
acq_id = 'MN16_RH_S26a_220912_high_res_region'

scope = 'iSPIM2'
dirname = acq_id
rootDir = os.path.join('/ispim2_data/workflow_data/iSPIM2', acq_id)

run_input = {
    "outputDir": f'/ACdata/processed/processed_samk/{acq_id}/',
    "rootDir": f'{rootDir}/',
    "ds_name": dirname,
    "mip_level": 3,
    "md_filename": f'{rootDir}/acqinfo_metadata.json',
    "consolidate_pos": True,
    "reverse_stitch": False,
    "deskew": False,
    "stitch_channel": 0,
    "stitch_json": "stitch-test.json",
    "stitch_final": "stitch-final.json",
    "state_json": "state-test.json",
    "nglink_name": "nglink-test.txt",
    "stitched_nglink": "stitched-nglink-test.txt",
    "stitched_state": "stitched-state.json"
}

md_input = {
    "rootDir": run_input['rootDir'],
    "fname": run_input['md_filename']
}
# md = parse_metadata.ParseMetadata(input_data=md_input).get_md()
# dsname = md["stripdirs"][0].split("_Pos")[0]
# run_input["ds_name"]=dsname


In [ ]:
# Parsing metadata
run_input["reverse_stitch"]=parse_metadata.ParseMetadata(input_data=md_input).get_direction()
deskew = 0

if run_input['deskew']:
    deskew = np.cos(parse_metadata.ParseMetadata(input_data=md_input).get_angle())

n_channels = parse_metadata.ParseMetadata(input_data=md_input).get_number_of_channels()
n_pos = parse_metadata.ParseMetadata(input_data=md_input).get_number_of_positions()

if n_pos == 0:
    print("No positions to process")

convert_input = {
    "input_dir": f"{run_input['rootDir']}",
    "output_dir": f"{run_input['outputDir']}",
    "max_mip": 5,
    "position_concurrency": 5
}

In [ ]:
# Converting to N5
if not os.path.isdir(convert_input['output_dir']):
    acquisition_dir_to_n5_dir.AcquisitionDirToN5Dir(input_data=convert_input).run()
else:
    print(f"Skipping conversion, {dirname} directory already exists")

In [ ]:
# Generating NG Link
state = {"showDefaultAnnotations": False, "layers": []}

overview_input={
    "run_input": run_input
}

create_overview.Overview(input_data=overview_input).run(n_channels, n_pos, dirname, deskew, state=state)